In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data <= 1/1
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W45`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W46`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W47`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W48`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W49`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W50`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W51`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W52`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W53`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W01`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W02`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W03`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.sort_values(by=['Week']).tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
11747,2021-01-15 13:05:22+00:00,2021-01-15 13:10:31+00:00,Production,0025 of 0047,A2,None,None,KL6,Saturday Assembly,Friday Morning,iXUsr_Gwen(7292),4,5.15,2021-03
11748,2021-01-15 12:44:57+00:00,2021-01-15 12:45:07+00:00,Production,0001 of 0026,H2,None,None,KL7,Saturday Assembly,Friday Morning,iXUsr_Strzeletz(6101),3,0.17,2021-03
64670,2021-01-15 12:55:54+00:00,2021-01-15 12:59:02+00:00,Preparation/Changeover,None,V4,None,None,KL13,None,Friday Morning,iXUsr_Siregar(6092),4,3.13,2021-03
1729,2021-01-15 12:51:47+00:00,2021-01-15 12:54:56+00:00,Break,None,P2,missing products,None,KL16,None,Friday Morning,iXUsr_Anne(7488),4,3.15,2021-03
43204,2021-01-14 06:40:30+00:00,2021-01-14 06:45:20+00:00,Production,0008 of 0016,R4,None,None,KL1,Wednesday Assembly,Thursday Morning,iXUsr_Siregar(6092),3,4.83,2021-03


In [5]:
data_in_use.shape

(68033, 14)